In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

In [34]:
np.random.seed(4)
X = np.random.uniform(-5, 5, 200)
epsilon = np.random.normal(0, 0.5, 200)
y = 2 * X**2 + X + epsilon

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, 1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)


In [35]:
class PolynomialRegressionModel(nn.Module):
    def __init__(self, degree):
        super(PolynomialRegressionModel, self).__init__()
        self.degree = degree
        self.poly = nn.Linear(degree, 1)

    def forward(self, x):
        poly_features = torch.cat([x**i for i in range(1, self.degree + 1)], dim=1)
        return self.poly(poly_features)

In [36]:
def train_model(model, X_train, y_train, X_val, y_val, num_epochs=1000, learning_rate=0.000001):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    train_losses = []
    val_losses = []

    for _ in range(num_epochs):
        optimizer.zero_grad()
        y_train_pred = model(X_train)
        train_loss = criterion(y_train_pred, y_train)
        train_losses.append(train_loss.item())
        train_loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_val_pred = model(X_val)
            val_loss = criterion(y_val_pred, y_val)
            val_losses.append(val_loss.item())
    
    return train_losses, val_losses

In [37]:
degrees = [5, 15]
train_errors = []
val_errors = []

for degree in degrees:
    model = PolynomialRegressionModel(degree)
    train_loss, val_loss = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor)
    train_errors.append(train_loss)
    val_errors.append(val_loss)

In [38]:
fig = go.Figure()

for i, degree in enumerate(degrees):
    fig.add_trace(go.Scatter(x=np.arange(1000), y=train_errors[i], mode='lines', name=f"Training Error (Degree {degree})", line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=np.arange(1000), y=val_errors[i], mode='lines', name=f"Validation Error (Degree {degree})", line=dict(color='red', dash='dash')))

fig.update_layout(
    title="Training and Validation Errors for Polynomial Regression",
    xaxis_title="Epochs",
    yaxis_title="Mean Squared Error (MSE)",
    template="ggplot2"
)

fig.show(config=dict(scrollZoom=True))